In [1]:
# library(glmnet)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
coef(summary(model))

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),0.913298868,0.1368576,6.673351556,2.500265e-11
Chromosomechr10,0.032339209,0.1317470,0.245464421,8.060969e-01
Chromosomechr11,0.103894156,0.1348980,0.770168393,4.412000e-01
Chromosomechr12,-0.146432213,0.1348400,-1.085969820,2.774923e-01
Chromosomechr13,0.212705900,0.1500070,1.417973071,1.561986e-01
Chromosomechr14,0.215474587,0.1463617,1.472206202,1.409652e-01
Chromosomechr15,0.049130882,0.1673433,0.293593316,7.690687e-01
Chromosomechr16,0.295829284,0.1627829,1.817323818,6.916755e-02
Chromosomechr17,-0.114925565,0.1692377,-0.679077697,4.970886e-01
Chromosomechr18,-0.346835512,0.1634299,-2.122227418,3.381865e-02


In [5]:
table(all_data$annotation)


not_transcribed protein_binding      regulatory     transcribed            UTR3 
         189937             111            4576          440024          167509 
           UTR5 
          89947 

In [1]:
library(dplyr)

args = commandArgs(trailingOnly=TRUE)
# tissue = args[1]
tissue = "blood"
# tissue_predOn = args[2]
tissue_predOn = "skin"
# model_name = args[3]
model_name = "model3"
tmp_file_path = "../../../"

if ((tissue_predOn == "liver")&&(tissue != "liver")){
    load(paste(tmp_file_path,"data/",tissue,"/objects/",model_name,"/",tissue,"_forLiver_model.RData",sep=""))#model
}else{
    load(paste(tmp_file_path,"data/",tissue,"/objects/",model_name,"/",tissue,"_model.RData",sep=""))#model
}
if ((tissue_predOn != "liver")&&(tissue == "liver")){
    load(paste(tmp_file_path,"data/",tissue_predOn,"/objects/",model_name,"/",tissue_predOn,"_forLiver_samples_sites_test.RData",sep=""))#sample_sites_test
    all_data <- read.csv(paste(tmp_file_path,"data/",tissue_predOn,"/dataframes/",model_name,"/",tissue_predOn,"_onLiver_all_data_readyForPrediction.csv",sep=""),header=TRUE)
}else{
    load(paste(tmp_file_path,"data/",tissue_predOn,"/objects/",model_name,"/",tissue_predOn,"_samples_sites_test.RData",sep=""))#sample_sites_test
    all_data <- read.csv(paste(tmp_file_path,"data/",tissue_predOn,"/dataframes/",model_name,"/",tissue_predOn,"_all_data_readyForPrediction.csv",sep=""),header=TRUE)
}
all_data$annotation <- gsub("protein_binding","not_transcribed",all_data$annotation)#there are so few protein binding sites that we may as well omit 
#rerplacing with "not_transcribed" is fine as they are the last level before non_transcribed in the annotation module     

probs <- predict.glm(model, all_data[sample_sites_test,], type="response")
probs_df <- data.frame(x = probs)

#binding the predicted proabilities to the OG data (but the test sites only)
probs_df <- probs_df %>%
    bind_cols(all_data[sample_sites_test,])

#renaming the glm_probs coloumn
colnames(probs_df ) <- replace(colnames(probs_df ), 1, "glm_probs")

#writing to file 
filename = paste(tmp_file_path,"data/",tissue,"/dataframes/",model_name,"/",tissue,"_on_",tissue_predOn,"_ProbabilityDf.csv",sep="")#this sep is for the filename string
write.csv(probs_df ,filename, row.names = FALSE)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels): factor annotation has new levels protein_binding
